In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

tree = Phylo.read('tree.nwk', 'newick')
print tree


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(tree)? (<ipython-input-1-a4b2d514a864>, line 6)

In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

tree = Phylo.read('tree.nwk', 'newick')
Phylo.draw(tree)

NameError: name 'Phylo' is not defined

In [51]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import Phylo
from Bio import Cluster
from Bio.Cluster import treecluster
import matplotlib.pyplot as plt
%matplotlib inline

# Create tree
tree = Phylo.read('tree.nwk', 'newick')
tree.ladderize() 
Phylo.draw_ascii(tree, file=None, column_width =125)


# Select one representative of each cluster - gene is determined using BLAST
record_dict =  SeqIO.to_dict(SeqIO.parse('seqs.aligned.fa','fasta'))
cluster_list = ['hu.39','pi.2','hu.17','rh.2','rh.40','rh.43','hu.14','hu.43','cy.2','rh.35','cy.3']
"""
## Cluster 1 - Adeno-associated virus isolate hu.39 capsid protein VP1 (cap) gene, complete cds
print(record_dict['hu.39'].format('fasta'))
## Cluster 2 - 
print(record_dict['pi.2'].format('fasta'))
## Cluster 3 - hu.17
print(record_dict['hu.39'].format('fasta'))
"""

# Calculating %AT and %CG in first total_pos positions of MSA cluster representatives
total_pos = 50
AT = [0]*total_pos
CG = [0]*total_pos

## Scanning all cluster representatives
for specimen in cluster_list:    
    sequence = record_dict[specimen].seq
    for pos in range(total_pos):
        if sequence[pos]=='A' or sequence[pos]=='T':
            AT[pos]+=1
        if sequence[pos]=='C' or sequence[pos]=='G':
            CG[pos]+=1

AT_f = [AT[i]/ (AT[i]+CG[i]) for i in range(total_pos)]
CG_f = [CG[i]/ (AT[i]+CG[i]) for i in range(total_pos)]


## Creating stacked bar plot
p1 = plt.bar([i for i in range(total_pos)], AT_f)
p2 = plt.bar([i for i in range(total_pos)], CG_f, bottom = AT_f)
plt.ylabel('%AT or %GC')
plt.xlabel('Position of Sequence')
plt.show()

cluster_list = ['hu.39','pi.2','hu.17','rh.2','rh.40','rh.43','hu.14','hu.43','cy.2','rh.35','cy.3']
cluster_number = [0,10,13,18,19,27,28,31,]


 , hu.39
 |
 | rh.50
 |
_, rh.49
 |
 |_ rh.57
 |
 | rh.51
 |
 , rh.53
 |
 | rh.64
 |
 , rh.52
 |
 , rh.61
 |
 | __ rh.58
 ||
 ||                                                    , pi.2
 ||                  __________________________________|
 ||                 |                                  |_ pi.3
 ||                 |                                  |
 ||                 |                                  | pi.1
 ||                 |
 ||                 |                   __ hu.17
 ||              ___|                 _|
 ||             |   |                | | hu.6
 ||             |   |    ____________|
 ||             |   |   |            |  , rh.10
 ||             |   |   |            |__|
  |             |   |   |               |, bb.1
  |             |   |___|               ||
  |             |       |                | bb.2
  |             |       |
  |             |       |       ___________ rh.2
  |             |       |      |
  |             |       |______|        

AttributeError: 'SeqRecord' object has no attribute 'index'